# PonCenter dealer assistant PoC

- Abdullah
- Bjorn
- Maud
- Norick

## Dataset diagram
<img height="400px" src="https://dev-icp.s3.eu-central-1.amazonaws.com/dataengineering/database_diagram.png">

In [3]:
# Packages installeren
import sys

# !pip install pandas
# !pip install transformers
# !pip install torch
# !pip install spacy
# !{sys.executable} -m pip install spacy
# !{sys.executable} -m spacy download nl_core_news_sm
!{sys.executable} -m pip uninstall -y transformers
!{sys.executable} -m pip uninstall -y huggingface-hub
!{sys.executable} -m pip uninstall -y tokenizers
!{sys.executable} -m pip install transformers==4.41.2

# Onze imports
from transformers import pipeline
import pandas as pd
import spacy
# import torch
# import dash
# import matplotlib.pyplot as plt
# import plotly.express as px
# import sys

# Configuratie
nlp = spacy.load('nl_core_news_sm')
pd.set_option('display.max_colwidth', None)

Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: huggingface-hub 0.36.0
Uninstalling huggingface-hub-0.36.0:
  Successfully uninstalled huggingface-hub-0.36.0
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Consider adding this directory to PATH or, if you prefer

In [4]:
# Vanuit Huggingface: https://huggingface.co/facebook/bart-large-mnli
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [5]:
# Inladen van datasets vanuit AWS S3 bucket

_S3Path = 'https://dev-icp.s3.eu-central-1.amazonaws.com/dataengineering/'

werkorders_df = pd.read_csv(_S3Path + 'workorders.csv')
berichten_df = pd.read_csv(_S3Path + 'remarks.csv')
koppeltabel_df = pd.read_csv(_S3Path + 'join.csv')
voertuigen_df = pd.read_csv(_S3Path + 'vehicles.csv')
# Overkoepelende tabel van onderdelen en arbeid regels
taken_df = pd.read_csv(_S3Path + 'works.csv') # kan je beter zien als pakketten (includeerd arbeid en onderdelen later in het process)

/tmp/ipykernel_420573/1685019421.py:5: DtypeWarning: Columns (7,12,50,58) have mixed types. Specify dtype option on import or set low_memory=False.
  werkorders_df = pd.read_csv(_S3Path + 'workorders.csv')
/tmp/ipykernel_420573/1685019421.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  berichten_df = pd.read_csv(_S3Path + 'remarks.csv')
/tmp/ipykernel_420573/1685019421.py:8: DtypeWarning: Columns (9,54,66,70) have mixed types. Specify dtype option on import or set low_memory=False.
  voertuigen_df = pd.read_csv(_S3Path + 'vehicles.csv')


## Kort overzicht analyze op de datasets

In [6]:
werkorders_df.head()

,id,car_vehicle_id,establishment,workshop,date_appointment,time_appointment,needs_follow_up_appointment,repeat_repair_deadline_date_time,follow_up_appointment_deadline_date_time,is_repeat_visit,...,remarks_workshop,remarks_warehouse,remarks_damage,total_labor,total_labor_finished,total_labor_need_approval,last_checkin_failure,created_by_sync,created,modified
0,745185,198965,Pon Center Amersfoort Volkswagen,Werkplaats,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,2024-10-15 11:51:53,2024-10-15 02:00:00,2024-11-06 00:26:50
1,534401,199008,Pon Center Amersfoort Skoda,Werkplaats,2023-01-10,12:15:00,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,NaN,2023-01-10 00:00:00,2023-07-09 06:37:47
2,176120,199020,Pon Center Amersfoort Skoda,Werkplaats,2020-08-14,13:15:00,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,NaN,2020-08-04 00:00:00,2020-08-04 00:00:00
3,263181,199020,Pon Dealer Amersfoort Seat,Werkplaats,2020-10-06,08:00:00,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,NaN,2020-10-01 00:00:00,2020-10-01 00:00:00
4,271416,199020,Pon Dealer Amersfoort Seat,Werkplaats,2020-08-13,08:00:00,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,NaN,2020-08-03 00:00:00,2020-08-03 00:00:00


In [7]:
berichten_df.head()

,id,workorder_id,remark_type,crm_partner_id,department,to_crm_partner_id,remarks,read,priority,remark_status,...,in_progress_datetime,postpone_datetime,postpone_reason,remark_parent_id,crm_partner_completed_id,completed_datetime,source,source_id,created,modified
0,104233,413943,diagnosis,420380,front_desk,50.0,NaN,NaN,action-required,handled,...,NaN,NaN,NaN,NaN,319048.0,2022-02-02 21:52:00,NaN,NaN,2022-02-02 20:03:27,2022-02-02 21:52:00
1,104234,413944,labor_signed,7,chief_workshop,50.0,De status is gewijzigd van 1 arbeidsregel(s) via de werkzaamheden tab,NaN,action-required,handled,...,NaN,NaN,NaN,NaN,319048.0,2022-02-02 20:44:12,NaN,NaN,2022-02-02 20:40:37,2022-02-02 20:44:12
2,104235,413944,diagnosis,319048,warehouse,50.0,test,NaN,action-required,handled,...,2022-02-02 20:43:49,NaN,NaN,NaN,319048.0,2022-02-02 20:43:52,NaN,NaN,2022-02-02 20:43:36,2022-02-02 20:43:52
3,104236,413944,diagnosis,319048,front_desk,50.0,test,NaN,action-required,handled,...,NaN,NaN,NaN,NaN,319048.0,2022-02-02 20:44:04,NaN,NaN,2022-02-02 20:43:52,2022-02-02 20:44:04
4,104237,413944,labor_signed,7,chief_workshop,50.0,De status is gewijzigd van 1 arbeidsregel(s) via de werkzaamheden tab,NaN,action-required,handled,...,NaN,NaN,NaN,NaN,842371.0,2022-02-02 21:07:38,NaN,NaN,2022-02-02 20:45:57,2022-02-02 21:07:38


In [8]:
print('Unique departments: ', berichten_df['department'].unique())
print('Unique types: ', berichten_df['remark_type'].unique())

Unique departments:  ['front_desk' 'chief_workshop' 'warehouse' 'warranty' 'workshop'
 'orderpicker' 'kcc' 'backoffice' 'telephone_operators']
Unique types:  ['diagnosis' 'labor_signed' 'warranty' 'check_delivery'
 'customer_feedback' 'other' 'make_appointment' 'autoline_exception'
 'partner_name_changed' 'part_availability' 'partner_changed'
 'customer_labor_statement' 'damage' 'workorder_finished'
 'maintenance_rejected' 'diagnosis_rejected' 'key_dropped'
 'customer_contact']


In [9]:
koppeltabel_df.head()

,workorder_id,work_id
0,172569,1
1,172569,1
2,175844,1
3,175844,1
4,175956,1


In [10]:
voertuigen_df.head()

,id,car_brand,car_model,coachwork,transmission,car_implementation_id,implementation_code,model_type,model,model_execution_group,...,bruto_bpm,catalog_price,datum_eerste_afgifte_nl,datum_eerste_toelating,description,is_exported,apk_expiration_date,note,created,modified
0,198887,SKOD,FABIA,STATIONWAGEN,A,152.0,NaN,6Y53F3,NaN,NaN,...,NaN,0.0,2003-07-10,2003-07-10,FABIA 1.4 Automaat Elegance 55kW / 75pk A,NaN,2026-05-19,NaN,2021-05-25 20:39:56,2025-04-22 01:30:30
1,199049,SKOD,OCTAVIA,STATIONWAGEN,A,179.0,NaN,1U5273,NaN,NaN,...,NaN,0.0,2001-01-06,2001-01-06,OCTAVIA 1.9 TDI Automaat Comfort 66kW / 90pk A,NaN,2018-01-11,NaN,2021-05-25 20:39:56,2025-02-05 01:34:04
2,199066,SKOD,OCTAVIA,STATIONWAGEN,A,181.0,NaN,1U52N3,NaN,NaN,...,NaN,0.0,2001-01-25,2001-01-25,OCTAVIA 2.0 Automaat Comfort 85kW / 115pk A,NaN,2020-11-16,NaN,2021-05-25 20:39:57,2022-07-15 08:16:04
3,199127,SKOD,FABIA,STATIONWAGEN,A,152.0,NaN,6Y53F3,NaN,NaN,...,NaN,0.0,2001-03-28,2001-03-28,FABIA 1.4 Automaat Elegance 55kW / 75pk A,NaN,2018-09-25,NaN,2021-05-25 20:39:57,2022-07-15 08:16:08
4,199130,SKOD,FABIA,HATCHBACK,A,152.0,NaN,6Y23F3,NaN,NaN,...,NaN,0.0,2001-03-30,2001-03-30,FABIA 1.4 Automaat Elegance 55kW / 75pk A,NaN,2026-03-20,NaN,2021-05-25 20:39:57,2025-03-11 01:30:38


In [11]:
taken_df.head(20)

,id,cid,description,created,modified
0,1,BARDIAG,Diagnose Occassion,2021-05-18 11:52:00,2025-11-09 01:30:02
1,8,PAF001,Nieuw Verhuur VW Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
2,9,PAF002,Nieuw Verhuur Audi Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
3,10,PAF003,Nieuw Verhuur LKW Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
4,11,PAF004,Nieuw Verhuur VW Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
5,12,PAF005,Nieuw Verhuur Seat Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
6,89,PDE001,Eigen Risico Pon Plus Garantie,2021-05-18 11:52:02,2025-11-09 01:30:02
7,90,PDE002,Ruitconcept voorruit 1e ster-reparatie. Let op! Foto's,2021-05-18 11:52:02,2025-11-09 01:30:02
8,91,PDE003,Ruitconcept voorruit tweede ster-reparatie. Let op! Foto's,2021-05-18 11:52:02,2025-11-09 01:30:02
9,92,PLAATWM,Plaat en Spuitwerk,2021-05-18 11:52:02,2025-11-09 01:30:02


# Datasets opschonen (cleanup)

### Berichten opschonen

In [1]:
berichten_df = berichten_df[berichten_df['remark_type'] == 'customer_contact'];

drop_columns = [
    'crm_partner_id',
    'to_crm_partner_id',
    'read',
    'remark_status',
    'postpone_datetime',
    'crm_partner_completed_id',
    'completed_datetime',
    'crm_partner_in_progress_id',
    'in_progress_datetime',
    'source_id',
    'created',
    'modified',
    'department'
]


berichten_df = berichten_df.drop(
    # Alleen kollommen die nu bestaan in de dataframe verwijderen
    columns=[col for col in drop_columns if col in berichten_df.columns.tolist()]
)

# klant berichten (1821 entries) ⊆ totale berichten (848127 entries); -99.79%
berichten_df.head()


NameError: name 'berichten_df' is not defined

In [13]:
# TODO .... werkorders_df, voertuigen_df en taken_df ook opschonen

# Model 1; Basic NLP text labeling

In [14]:
berichten_df.groupby('workorder_id').head()

,id,workorder_id,remark_type,remarks,priority,postpone_reason,remark_parent_id,source
804543,1247972,796945,customer_contact,Klant bericht test,medium,NaN,NaN,NaN
804544,1247973,796945,customer_contact,Afbeelding klantportaal test,medium,NaN,NaN,NaN
804585,1248388,794612,customer_contact,"Goedemorgen,\n\nIk heb bij het inchecken aangegeven geen vervangend vervoer nodig te hebben. Bij nader inzien zou ik graag een fiets willen lenen om naar huis te rijden. Is dit alsnog mogelijk?\n\nMvg, Kristian",medium,NaN,NaN,NaN
804633,1248785,798678,customer_contact,Wat is de situzatie op dit moment? Auto is sinds. Dinsdag in onderzoek. Graag wat duidelijkheid,medium,NaN,NaN,NaN
804674,1249080,797158,customer_contact,"Hoi, ik zie dat het voertuig nog steeds staat op 'ingecheckt'. Is dat de status of zit er wat vertraging in de software? Ik vraag het even ivm afspraken eind vd middag en de inschatting of mijn auto dan al klaar is.",medium,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
848038,1588049,880885,customer_contact,Hoi,medium,NaN,NaN,NaN
848062,1588230,880015,customer_contact,Verhuizing svp Doorne in het system. Dank,medium,NaN,NaN,NaN
848063,1588232,875278,customer_contact,Willen jullie de afspraak even annuleren,medium,NaN,NaN,NaN
848066,1588253,873601,customer_contact,Is er wel al gekeken door de juiste afdeling naar de melding die de auto heeft? Of staat hij daar nog ‘op de wachtlijst’.,medium,NaN,NaN,NaN


In [15]:
taken_df.head(100)

,id,cid,description,created,modified
0,1,BARDIAG,Diagnose Occassion,2021-05-18 11:52:00,2025-11-09 01:30:02
1,8,PAF001,Nieuw Verhuur VW Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
2,9,PAF002,Nieuw Verhuur Audi Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
3,10,PAF003,Nieuw Verhuur LKW Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
4,11,PAF004,Nieuw Verhuur VW Kost.Rijkl.Maken,2021-05-18 11:52:00,2025-11-09 01:30:02
...,...,...,...,...,...
95,314,A 982758428,S-Tronic olie verversen,2021-05-18 16:24:46,2021-05-18 16:24:46
96,315,A 986060241,Interieurfilter vervangen,2021-05-18 16:24:46,2021-05-18 16:24:46
97,316,A 463627681,Remblokken vervangen (Voor),2021-05-18 16:24:46,2021-05-18 16:24:46
98,317,A 463836957,Remblokken vervangen (Achter),2021-05-18 16:24:46,2021-05-18 16:24:46


In [16]:
# Optie 1; met NLP mapping van token en taken id

# Van onderdelen en arbeid regels tokenizeren en verwijzen naar lijst van cids
# i.e. {token: [cid1, cid2, cid3 .....], ...}: token_cid_map

token_cid_map = {}
for index, row in taken_df.iterrows():
    cid = row['cid']
    doc = nlp(str(row['description']))

    for nouns in doc.noun_chunks:
        for token in nouns:
          if token.lower_ in ['gebruikte']:
              continue
          if not token.is_punct and not token.is_space and not token.is_stop and token.is_alpha:
              if token.lower_ not in token_cid_map:
                  token_cid_map[token.lower_] = [cid]
              else:
                token_cid_map[token.lower_].append(cid)

print(token_cid_map)



{'verhuur': ['PAF001', 'PAF002', 'PAF003', 'PAF004', 'PAF005'], 'garantie': ['PDE001', 'PDE004'], 'ruitconcept': ['PDE002', 'PDE003'], 'tweede': ['PDE003'], 'velg': ['PLAATWM3'], 'spuiten': ['PLAATWM3'], 'kosten': ['PON001', 'PON002', 'PON050', 'PON060', 'PON004', 'PON008', 'PON061', 'PON009'], 'lease': ['PON002', 'PON052', 'PON173', 'PON936', 'PON369', 'PON370', 'PON008', 'PON373'], 'service': ['PON003'], 'nieuwe': ['PON015'], 'auto': ['PON015', 'Pon214', 'AP07HB00001', 'LP07HB00001', 'H 3410D4454', 'DP07HB00001', 'HP07HB00001', 'SP07HB00001', 'SP07HB00005', 'H 3410D4450', 'H 3410D815D', 'H 3410D5A08', 'MUN115', 'H 3410D8F32', 'H 3857D7BF5', 'H 3410D8F37', 'H 3410D8F3A', 'H 3410D5A07', 'H 3410D446D', 'H 3410D8F30', 'H 3410D9273'], 'opgelegde': ['PON090'], 'letters': ['PON090'], 'witte': ['PON091'], 'kentekenplaat': ['PON091'], 'montage': ['PON160', 'PON161', 'PON162', 'PON163', 'PON170', 'PON171', '8X005463000001', '8V009216000001', 'PON194', 'ZR 000104CCA00001', 'PT0100152', 'ZS 5JA2

In [28]:
# TODO: De tokens die het meeste id's zullen warschijnlijk generieke tokens zijn, die kunnen we hoogwaarschijnlijk wegehalen anders gaan ze zichzelf hoger presteren

for key in sorted(token_cid_map, key=lambda k: len(token_cid_map[k]), reverse=True)[:100]:
    print(key, len(token_cid_map[key]))


blokken 1481
remblokken 1155
bougies 871
ruitenwissers 727
voorruit 706
verversen 681
remvloeistof 647
olie 435
buitenzijde 407
schokdempers 379
aandrijfashoezen 370
trekhaak 361
mattenset 313
uitgebreide 240
inspecties 240
trekhaakset 177
modificatie 170
vaste 160
afneembare 144
gloeibougies 141
zwenkbare 130
incl 126
aanv 125
wzh 124
inch 113
compressor 105
waterpomp 103
controleren 103
partikelfilter 88
winterset 88
schuifdeur 88
koppelstang 82
lichtmetalen 76
schoonmaken 65
waterkast 63
achterbumper 62
wandbekleding 59
vervanging 54
dakdragers 50
navigatie 49
distributieriem 46
kofferbakmat 44
versnellingsbak 43
ringen 41
achterzijde 40
dubbele 39
gasinstallatie 38
dubbelzijdige 38
kabelset 37
achteras 35
adblue 34
crafter 33
montage 32
rubberen 32
onderhoud 31
banden 29
opschrift 28
zwart 27
stijl 27
jazz 26
velours 26
houtnerf 24
kofferbakinleg 24
veloursmatten 24
cabine 23
set 22
variabele 22
auto 21
pakket 21
wegklapbare 21
tussenschot 21
trommelremmen 21
telematica 21
zomerset

In [17]:
# Voorbeeld out of the box Facebook classifier
classifier('Graag bestel ik een fles olie om te kunnen bijvullen komende maanden!Bvd Greet', ['olie', 'banden', 'vullen'])

{'sequence': 'Graag bestel ik een fles olie om te kunnen bijvullen komende maanden!Bvd Greet',
 'labels': ['olie', 'vullen', 'banden'],
 'scores': [0.7991437315940857, 0.1443922370672226, 0.056464072316884995]}

In [18]:
# Het combineren van alle werkorder kalnt berichten in een string dat naast een werkorder id
combined_remarks_df = berichten_df.groupby('workorder_id')['remarks'].apply(lambda x: '; '.join(x.dropna())).reset_index()
combined_remarks_df.head()

,workorder_id,remarks
0,749972,"Prima, hadden we wel verwacht"
1,753977,"Goededag, er schoot me iets te binnen. Over een drempel gaan kraken de veren erg. Is dit te verhelpen?"
2,756365,Ik wilde nog zeggen dat de zomerbouten bij de bijrijdersstoel liggen. Ik kan u telefonisch niet bereiken.
3,757629,Ligt mij alstublieft in over wat en welk voertuig het gaat ??\n\n; Ok duidelijk
4,757720,Dat is snel :) ik ga ff wat verzetten dan probeer ik zo jullie kant op de komen :)


In [19]:
#  Dit classifier is goed om te sorteren!!!

example = combined_remarks_df[combined_remarks_df['workorder_id'] == 878757].iloc[0];

classifier(example['remarks'], list(token_cid_map.keys()))

# for index, row in combined_remarks_df.head().iterrows():
#     workorder = row['workorder_id']
#     remarks = row['remarks']
#     result = classifier(remarks, token_cid_map.keys())
#     print(result)
#     break;

{'sequence': 'Als jullie de slotbouten willen vervangen graag. \nDank\nMaurice',
 'labels': ['vervanging',
  'slotbouten',
  'connected',
  'verversen',
  'vervangen',
  'variant',
  'multifunctioneel',
  'grijs',
  'vering',
  'opgelegde',
  'vervoer',
  'inclu',
  'multi',
  'connect',
  'set',
  'incl',
  'lenden',
  'varianten',
  'var',
  'verz',
  'bouten',
  'deur',
  'verzwaring',
  'gaten',
  'activeren',
  'aanv',
  'blades',
  'wiellager',
  'bevestig',
  'let',
  'tr',
  'diverse',
  'aanvullen',
  'modificatie',
  'upgrade',
  'geïntegreerde',
  'duo',
  's',
  'vibraties',
  'bodem',
  'wieldoppen',
  'accubehuizing',
  'remvoering',
  'uitvouwbare',
  'gl',
  'transparant',
  'passat',
  'alarm',
  'chek',
  'behoeve',
  'bod',
  'bevestigin',
  'batterijen',
  'uitgebreide',
  'bekleding',
  'balanceren',
  'controleren',
  'v',
  'versie',
  'vlag',
  'lekt',
  'ruit',
  'i',
  'gebruik',
  'allesdrager',
  'luchtvering',
  'achtermat',
  'btw',
  'lage',
  'l',
  'bel

In [20]:
# TODO
# damage_keywords = [
#     'schade', 'kapot', 'defect', 'barst', 'lek', 'krassen', 'deuk', 'probleem',
#     'ruitschade', 'motorstoring', 'remmen', 'banden', 'lichten', 'ruit', 'spiegel',
#     'bumper', 'vleugel', 'portier', 'airco', 'koppeling', 'versnellingsbak', 'uitlaat',
#     'roest', 'stuur', 'ophanging', 'accu', 'startprobleem', 'koelvloeistof', 'olie',
#     'abs', 'airbag', 'waterpomp', 'distributieriem', 'turbo', 'injector', 'sensor',
#     'electronica', 'kabelboom', 'bekabeling', 'bekleding', 'interieur', 'exterieur',
#     'lak', 'voorruit', 'achterruit', 'zijruit', 'schuifdak', 'velg', 'band'
# ]

# # Re-initialize damage_analysis_df to ensure a clean state for processing
# damage_analysis_results_reinit = []
# for workorder_id in combined_remarks_df['workorder_id']:
#     damage_analysis_results_reinit.append({
#         'workorder_id': workorder_id,
#         'implies_car_damage': False,
#         'damaged_parts_list': []
#     })
# damage_analysis_df = pd.DataFrame(damage_analysis_results_reinit)

# # First pass: Apply keyword matching to populate 'implies_car_damage' and 'damaged_parts_list'
# for index, row in combined_remarks_df.iterrows():
#     workorder_id = row['workorder_id']
#     remarks_text = str(row['remarks']).lower()

#     found_keywords = []
#     for keyword in damage_keywords:
#         if keyword in remarks_text:
#             found_keywords.append(keyword)

#     if found_keywords:
#         idx_to_update = damage_analysis_df.index[damage_analysis_df['workorder_id'] == workorder_id].tolist()[0]
#         damage_analysis_df.at[idx_to_update, 'implies_car_damage'] = True
#         damage_analysis_df.at[idx_to_update, 'damaged_parts_list'] = list(set(found_keywords))

# # Second pass: Apply NER for workorders where car damage is implied
# for index, row in combined_remarks_df.iterrows():
#     workorder_id = row['workorder_id']
#     remarks_text = str(row['remarks']).lower()

#     current_damage_row_idx = damage_analysis_df.index[damage_analysis_df['workorder_id'] == workorder_id].tolist()

#     if current_damage_row_idx:
#         idx_to_update = current_damage_row_idx[0]
#         if damage_analysis_df.at[idx_to_update, 'implies_car_damage']:
#             doc = nlp(remarks_text)
#             extracted_parts = set(damage_analysis_df.at[idx_to_update, 'damaged_parts_list']) # Start with already found keywords

#             for ent in doc.ents:
#                 if ent.text.lower() in damage_keywords: # Check if entity is one of our car parts
#                     extracted_parts.add(ent.text.lower())

#             # Update the 'damaged_parts_list' with unique extracted parts
#             damage_analysis_df.at[idx_to_update, 'damaged_parts_list'] = list(extracted_parts)

# # Display the first few entries to verify the updated structure
# print(damage_analysis_df.head())

# # # Display some basic information about the updated DataFrame
# # print(damage_analysis_df.info())

# # # Optional: Check how many entries have 'implies_car_damage' as True
# # print(f"Number of workorders implying car damage: {damage_analysis_df['implies_car_damage'].sum()}")




In [21]:
damage_analysis_df[damage_analysis_df['implies_car_damage']]

NameError: name 'damage_analysis_df' is not defined

In [ ]:
# damage_analysis_df[damage_analysis_df['implies_car_damage']]
# pd.set_option('display.max_colwidth', None)
combined_remarks_df[combined_remarks_df['workorder_id'] == 787826]

,workorder_id,remarks
45,787826,Graag bestel ik een fles olie om te kunnen bijvullen komende maanden!Bvd Greet


- We hebben nu een lijst van tokens -> part cids
- we kunnen nu op basis van een klant bericht text labelen en daarvan part cids ophalen

Omdat deze part cids elke type voertuig bevatten gaan we hier ook op basis van voertuig verder moeten filteren, uiteindelijk zullen we misschien lijst van 5 of 10 onderdelen of arbeid regels hebben.

op de resenterende x onderdelen en of arbeid regels kunnen we op een slimmme manier sorteren (Bonus denk ik)